In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json

import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

from statsbombpy import sb


/Users/thomassinapi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
events = pd.read_csv('events.csv')

/var/folders/hy/27t6v09s54x9lv_68y2yk65w0000gn/T/ipykernel_90563/1763814053.py:1: DtypeWarning: Columns (46,52,57,61,63,68,69,72,98,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,120,121,122,124,125,126,127,128,129,130,131,132,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv('events.csv')


In [3]:
events.shape

(2360009, 151)

In [36]:
euro = events[events['competition_season'] == "UEFA Euro_2020"]
cdm = events[events['competition_season'] == "FIFA World Cup_2022"]
cdm_final = cdm[cdm['match_date'] == "2022-12-18"]

In [15]:
cdm.shape


(234652, 151)

In [41]:
import numpy as np

# Tri du dataframe par index pour s'assurer que les événements sont ordonnés dans le temps
df = cdm_final.sort_index()

# Créer une colonne 'PSV' avec des valeurs par défaut de 0
df['PSV'] = 0

# Identifier les indices des tirs dans le dataframe
shoot_indices = df[df['type.name'] == 'Shot'].index

# Calculer le PSV pour chaque séquence de possession
for idx in shoot_indices:
    possession_idx = df.loc[:idx, 'possession'].max()
    possession_events = df[df['possession'] == possession_idx]

    # Calculer le PSV pour la séquence de possession
    total_xG = possession_events['shot.statsbomb_xg'].sum()
    df.loc[df['possession'] == possession_idx, 'PSV'] = total_xG



,match_id,competition_season,match_date,match_outcome,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,location,player.id,player.name,position.id,position.name,pass.recipient.id,pass.recipient.name,pass.length,pass.angle,pass.height.id,pass.height.name,pass.end_location,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,carry.end_location,pass.outcome.id,pass.outcome.name,counterpress,under_pressure,dribble.outcome.id,dribble.outcome.name,dribble.overrun,clearance.right_foot,clearance.body_part.id,clearance.body_part.name,clearance.head,pass.cross,pass.cut_back,interception.outcome.id,interception.outcome.name,ball_receipt.outcome.id,ball_receipt.outcome.name,off_camera,out,duel.type.id,duel.type.name,pass.aerial_won,pass.switch,clearance.aerial_won,duel.outcome.id,duel.outcome.name,ball_recovery.recovery_failure,foul_committed.card.id,foul_committed.card.name,pass.through_ball,pass.technique.id,pass.technique.name,clearance.left_foot,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.type.id,goalkeeper.type.name,goalkeeper.body_part.id,goalkeeper.body_part.name,pass.assisted_shot_id,pass.shot_assist,shot.statsbomb_xg,shot.end_location,shot.key_pass_id,shot.type.id,shot.type.name,shot.outcome.id,shot.outcome.name,shot.technique.id,shot.technique.name,shot.body_part.id,shot.body_part.name,shot.freeze_frame,goalkeeper.technique.id,goalkeeper.technique.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.end_location,foul_won.defensive,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,shot.first_time,shot.one_on_one,pass.outswinging,miscontrol.aerial_won,pass.inswinging,pass.deflected,pass.goal_assist,clearance.other,pass.no_touch,block.offensive,block.deflection,injury_stoppage.in_chain,block.save_block,foul_committed.advantage,foul_won.advantage,dribble.nutmeg,foul_committed.type.id,foul_committed.type.name,shot.aerial_won,foul_committed.offensive,bad_behaviour.card.id,bad_behaviour.card.name,shot.deflected,shot.open_goal,ball_recovery.offensive,pass.straight,goalkeeper.punched_out,pass.miscommunication,shot.saved_to_post,goalkeeper.shot_saved_to_post,50_50.outcome.id,50_50.outcome.name,shot.saved_off_target,goalkeeper.shot_saved_off_target,player_off.permanent,dribble.no_touch,foul_committed.penalty,foul_won.penalty,shot.follows_dribble,goalkeeper.success_in_play,goalkeeper.lost_in_play,goalkeeper.success_out,shot.redirect,goalkeeper.lost_out,goalkeeper.penalty_saved_to_post,half_start.late_video_start,goalkeeper.saved_to_post,half_end.early_video_end
1125069,3869685,FIFA World Cup_2022,2022-12-18,0,6d527ebc-a948-4cd8-ac82-daced35bb715,771,1,00:22:24.114,22,24,32,0.625635,16,Shot,779,Argentina,5,Other,779,Argentina,NaN,NaN,['c9b8e568-dcdc-4302-9683-0e9e9a55a42a'],"[108.0, 40.0]",5503.0,Lionel Andrés Messi Cuccittini,17.0,Right Wing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.783500,"[120.0, 41.8, 0.2]",NaN,88.0,Penalty,97.0,Goal,93.0,Normal,38.0,Left Foot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128689,3869685,FIFA World Cup_2022,2022-12-18,0,2a15edb5-2a54-41d5-afe9-87e447389b18,4391,5,00:01:02.191,121,2,249,0.843986,16,Shot,779,Argentina,5,Other,779,Argentina,NaN,NaN,['571f2328-3a11-4aa6-b344-0f281668a2e7'],"[108.1, 40.1]",5503.0,Lionel Andrés Messi Cuccittini,22.0,Right Center Forward,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [70]:
# afficher lignes de chque changement  possession

# Supposons que votre DataFrame s'appelle df
# Triez le DataFrame par 'timestamp' pour vous assurer que les événements sont dans l'ordre chronologique

# Ajoutez une colonne 'change_of_possession' qui indique les changements de possession
df['change_of_possession'] = df['possession'].ne(df['possession'].shift())

# Filtrer les lignes où il y a un changement de possession
change_of_possession_df = df[df['change_of_possession']]

# Afficher les lignes correspondant à chaque changement de possession
df[["minute","possession", "player.name","change_of_possession", "type.name", "PSV", "shot.statsbomb_xg"]].iloc[4050:4110]

#change_of_possession_df[["minute","second","possession", "player.name","change_of_possession", "type.name", "PSV", "shot.statsbomb_xg"]].head(60)


,minute,possession,player.name,change_of_possession,type.name,PSV,shot.statsbomb_xg
1128349,107,228,Leandro Daniel Paredes,True,Ball Recovery,0.592476,NaN
1128350,107,228,Leandro Daniel Paredes,False,Carry,0.592476,NaN
1128351,107,228,Leandro Daniel Paredes,False,Pass,0.592476,NaN
1128352,107,228,Gonzalo Ariel Montiel,False,Ball Receipt*,0.592476,NaN
1128353,107,228,Gonzalo Ariel Montiel,False,Carry,0.592476,NaN
1128354,107,228,Gonzalo Ariel Montiel,False,Pass,0.592476,NaN
1128355,107,228,Lautaro Javier Martínez,False,Ball Receipt*,0.592476,NaN
1128356,107,228,Lautaro Javier Martínez,False,Pass,0.592476,NaN
1128357,107,228,Lionel Andrés Messi Cuccittini,False,Ball Receipt*,0.592476,NaN
1128358,107,228,Lionel Andrés Messi Cuccittini,False,Carry,0.592476,NaN
